<a href="https://colab.research.google.com/github/noushadkhan01/Projects/blob/master/Hackathons/Analytics%20Vidhya/Build%20a%20Recommender%20Engine%20Powered%20by%20IBM%20Cloud/Recommender%20Engine%20Powered%20by%20IBM%20Cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [40]:
train = pd.read_csv('https://bit.ly/32Okxx9', parse_dates = ['InvoiceDate'])
train.head()

,CustomerID,InvoiceNo,Quantity,InvoiceDate,UnitPrice,Country,StockCode
0,27270,27270,7,2010-01-12 08:26:00,2.55,PX,85123AY
1,27270,27270,7,2010-01-12 08:26:00,3.39,PX,71053R
2,27270,27270,9,2010-01-12 08:26:00,2.75,PX,84406BH
3,27270,27270,7,2010-01-12 08:26:00,3.39,PX,84029GV
4,27270,27270,7,2010-01-12 08:26:00,3.39,PX,84029EX


In [41]:
train.shape

(330575, 7)

In [42]:
test = pd.read_csv('https://bit.ly/30SBLrj', parse_dates = ['InvoiceDate'])
test.head()

,Country,CustomerID,InvoiceDate,InvoiceNo,Quantity,StockCode,UnitPrice
0,PX,127269,2010-01-12 08:28:00,127269,7,22633V,1.85
1,PX,227268,2010-01-12 08:34:00,227268,38,84879M,1.69
2,PX,227268,2010-01-12 08:34:00,227268,7,22748P,2.10
3,PX,227268,2010-01-12 08:34:00,227268,9,22749K,3.75
4,PX,227268,2010-01-12 08:34:00,227268,2,22622G,9.95


In [4]:
train.shape, test.shape

((330575, 7), (103097, 7))

In [11]:
train.isnull().sum().sort_values(ascending = False)[:2]

StockCode    0
Country      0
dtype: int64

In [12]:
test.isnull().sum().sort_values(ascending = False)[:2]

UnitPrice    0
StockCode    0
dtype: int64

**As we can see train and test dataset have differend order for columns so we need to reorder this**

In [13]:
train.columns

Index(['CustomerID', 'InvoiceNo', 'Quantity', 'InvoiceDate', 'UnitPrice',
       'Country', 'StockCode'],
      dtype='object')

In [14]:
test.columns

Index(['Country', 'CustomerID', 'InvoiceDate', 'InvoiceNo', 'Quantity',
       'StockCode', 'UnitPrice'],
      dtype='object')

In [46]:
test = test[train.columns]
test.head()

,CustomerID,InvoiceNo,Quantity,InvoiceDate,UnitPrice,Country,StockCode
0,127269,127269,7,2010-01-12 08:28:00,1.85,PX,22633V
1,227268,227268,38,2010-01-12 08:34:00,1.69,PX,84879M
2,227268,227268,7,2010-01-12 08:34:00,2.10,PX,22748P
3,227268,227268,9,2010-01-12 08:34:00,3.75,PX,22749K
4,227268,227268,2,2010-01-12 08:34:00,9.95,PX,22622G


In [16]:
test.columns

Index(['CustomerID', 'InvoiceNo', 'Quantity', 'InvoiceDate', 'UnitPrice',
       'Country', 'StockCode'],
      dtype='object')

**no of unique countries**

In [33]:
train.Country.nunique()

38

**no. of unique products**

In [31]:
train.StockCode.nunique()

3810

In [44]:
test.InvoiceDate.dtype

dtype('<M8[ns]')

**invoice date**

In [45]:
train.InvoiceDate.min(), train.InvoiceDate.max()

(Timestamp('2010-01-12 08:26:00'), Timestamp('2011-12-10 17:19:00'))

drop columns those are not important to the model

In [0]:
def check_invoice_count_per_customer(data):
  id_g = data.groupby('CustomerID')
  yes = 0
  for i, j in id_g:
    uinvoice = j.InvoiceNo.nunique()
    if uinvoice > 1:
      print(i)
      print(uinvoice, '\n\n')
      yes += 1
  if not yes:
    print('none of the customers have bought more than 1 time')

In [27]:
check_invoice_count_per_customer(train)

none of the customers have bought more than 1 time


In [28]:
check_invoice_count_per_customer(test)

none of the customers have bought more than 1 time
